In [1]:
from sagas.ofbiz.builder import oc, finder, desc_model, abbrev
desc_model("SaMovie", False)

SA_MOVIE Movie Entity None
+------------------+----------------+-------------------+
| name             | type           | string            |
|------------------+----------------+-------------------|
| overview         | description    | OVERVIEW          |
| releaseDate      | date           | RELEASE_DATE      |
| movieId          | id             | MOVIE_ID          |
| video            | indicator      | VIDEO             |
| title            | name           | TITLE             |
| originalLanguage | id             | ORIGINAL_LANGUAGE |
| originalTitle    | name           | ORIGINAL_TITLE    |
| popularity       | floating-point | POPULARITY        |
| voteCount        | numeric        | VOTE_COUNT        |
| backdropPath     | url            | BACKDROP_PATH     |
| posterPath       | url            | POSTER_PATH       |
+------------------+----------------+-------------------+
+-------------------+--------+-------------------+
| name              | type   | string            |
|

In [2]:
Long = oc.gateway.jvm.Long
ValueHelper=oc.gateway.jvm.com.sagas.generic.ValueHelper
val = oc.delegator.makeValue("SaMovie")
ValueHelper.setDate(val, "releaseDate", "2014-10-22")

In [10]:
from json_utils import read_json_file
data_root=read_json_file("./data/movies_f.json")
genres_root=read_json_file("./data/movie_genres_f.json")

genres_map={}
for genre in genres_root['genres']:
    genres_map[genre['id']]=genre['name']

print(data_root["page"], "/", data_root["total_pages"])
movies=data_root["results"]
for movie in movies:
    print(movie['title'], movie['genre_ids'])
    for id in movie['genre_ids']:
        print("\t", genres_map[id])

1 / 55
John Wick [28, 53]
	 Action
	 Thriller
Captain America: Civil War [12, 28, 878]
	 Adventure
	 Action
	 Science Fiction
Batman v Superman: Dawn of Justice [28, 12, 14]
	 Action
	 Adventure
	 Fantasy
Deadpool [28, 12, 35]
	 Action
	 Adventure
	 Comedy
Doctor Strange [28, 12, 14, 878]
	 Action
	 Adventure
	 Fantasy
	 Science Fiction
Mad Max: Fury Road [28, 12, 878, 53]
	 Action
	 Adventure
	 Science Fiction
	 Thriller
Jurassic World [28, 12, 878, 53]
	 Action
	 Adventure
	 Science Fiction
	 Thriller
X-Men: Apocalypse [28, 12, 878, 14]
	 Action
	 Adventure
	 Science Fiction
	 Fantasy
Now You See Me 2 [28, 35, 53]
	 Action
	 Comedy
	 Thriller
Star Wars: The Force Awakens [28, 12, 878, 14]
	 Action
	 Adventure
	 Science Fiction
	 Fantasy
Sicario [28, 80, 18, 9648, 53]
	 Action
	 Crime
	 Drama
	 Mystery
	 Thriller
Spectre [28, 12, 80]
	 Action
	 Adventure
	 Crime
Underworld: Blood Wars [28, 14, 27, 18]
	 Action
	 Fantasy
	 Horror
	 Drama
Kingsman: The Secret Service [80, 35, 28, 12]
	 

In [11]:
Long = oc.gateway.jvm.Long
ValueHelper=oc.gateway.jvm.com.sagas.generic.ValueHelper

def indicator(val):
    return 'Y' if val else 'N'
def create_or_store(entity, movie):
    val = oc.delegator.makeValue(entity)
    val.set("movieId", str(movie["id"]))
    # val.set("voteCount", movie['vote_count'])
    ValueHelper.setNumericValue(val, "voteCount", movie['vote_count'])
    val.set('video', indicator(movie['video']))
    val.set('title', movie['title'])
    val.set('popularity', movie['popularity'])
    val.set('posterPath', movie['poster_path'])
    val.set('overview', abbrev(movie['overview'], 250))
    # val.set('releaseDate', movie['release_date'])
    ValueHelper.setDate(val, "releaseDate", movie['release_date'])
    
    return oc.delegator.createOrStore(val)

def store_movie_genres(genre):
    val = oc.delegator.makeValue("SaMovieGenres")
    val.set("movieGenresId", str(genre["id"]))
    val.set("movieGenresName", genre["name"])
    return oc.delegator.createOrStore(val)

def set_movie_genres(movie_id, genre_ids):
    for gid in genre_ids:
        val = oc.delegator.makeValue("SaMovieGenresAppl")
        val.set("movieId", str(movie_id))
        val.set("movieGenresId", str(gid))
        oc.delegator.createOrStore(val)
    
for genre in genres_root['genres']:
    store_movie_genres(genre)
    
for movie in movies:
    result=create_or_store("SaMovie", movie)
    set_movie_genres(movie["id"], movie['genre_ids'])
    print(result.get("title"))

John Wick
Captain America: Civil War
Batman v Superman: Dawn of Justice
Deadpool
Doctor Strange
Mad Max: Fury Road
Jurassic World
X-Men: Apocalypse
Now You See Me 2
Star Wars: The Force Awakens
Sicario
Spectre
Underworld: Blood Wars
Kingsman: The Secret Service
Terminator 2: Judgment Day
The Hunger Games: Mockingjay - Part 2
The Purge: Election Year
Allegiant
Dragon Ball Z: Resurrection 'F'
Suicide Squad


In [6]:
for mov in oc.all("SaMovie"):
    print(mov.get("title"))

John Wick
Captain America: Civil War
Batman v Superman: Dawn of Justice
Deadpool
Doctor Strange
Mad Max: Fury Road
Jurassic World
X-Men: Apocalypse
Now You See Me 2
Star Wars: The Force Awakens
Sicario
Spectre
Underworld: Blood Wars
Kingsman: The Secret Service
Terminator 2: Judgment Day
The Hunger Games: Mockingjay - Part 2
The Purge: Election Year
Allegiant
Dragon Ball Z: Resurrection 'F'
Suicide Squad
